In [49]:
"""
================================================================================
NOTEBOOK: GRAFICOS ESTRATEGICOS - ANALISE MEDIACAO BANCARIA AGIBANK
================================================================================

Objetivo: Criar 10 graficos estrategicos para analise comparativa Agibank vs Mercado

Autor: Equipe de Analise de Dados
Data: 25/02/2026
Status: Pronto para Producao

Graficos:
1. Mapa de Calor Brasil
2. Top Municipios SP (Volume + Taxa Normalizada)
3. Top Instituicoes Financeiras SP
4. Analise Campinas (Agibank vs Mercado)
5. Pontos Fracos Setor Financeiro
6. Benchmarks Setor Financeiro
7. Pontos Fracos Agibank
8. Pontos Fortes Agibank
9. Matriz de Oportunidades
10. Plano de Acao

================================================================================
"""

print("=" * 100)
print("NOTEBOOK: GRAFICOS ESTRATEGICOS - ANALISE AGIBANK")
print("=" * 100)

NOTEBOOK: GRAFICOS ESTRATEGICOS - ANALISE AGIBANK


In [50]:
# CONFIGURACAO DO AMBIENTE E IMPORTACOES

import sys
from pathlib import Path

# Obter diretorio atual
caminho_atual = Path.cwd()

# Subir 2 niveis para chegar na raiz do projeto
raiz_projeto = caminho_atual.parent.parent

# Adicionar raiz do projeto ao path
sys.path.insert(0, str(raiz_projeto))

print(f"Diretorio atual: {caminho_atual}")
print(f"Raiz do projeto: {raiz_projeto}")
print(f"Caminho adicionado ao sys.path")

# Verificar se a pasta lib existe
caminho_lib = raiz_projeto / 'lib'
print(f"\nPasta 'lib' encontrada em: {caminho_lib}")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px        
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from datetime import datetime
import json
import urllib.request


print("\nBibliotecas basicas importadas!")

# Importar modulos customizados
from lib.carregamento import (
    carregar_base_silver,
    carregar_base_gold_sp,
    carregar_base_agibank,
    carregar_base_setorial,
    carregar_base_filtrada,
    listar_arquivos_disponiveis,
    info_base
)

from lib.cores import (
    aplicar_tema_agibank,
    configurar_plotly,
    CORES_AGIBANK,
    PALETA_CATEGORICA,
    PALETA_AZUL,
    PALETA_VERDE,
    PLOTLY_COLORS,
    PLOTLY_PALETTE,
    PLOTLY_SCALE_AZUL,
    PLOTLY_SCALE_VERDE
)

from lib.visualizacoes import (
    grafico_barras,
    grafico_linha,
    grafico_pizza,
    grafico_boxplot,
    grafico_heatmap,
    grafico_distribuicao,
    grafico_comparativo_barras
)

print("Modulos customizados importados!")

# Configuracoes
warnings.filterwarnings('ignore')
np.random.seed(42)

np.set_printoptions(
    precision=2,
    suppress=True,
    linewidth=120,
    edgeitems=5,
    threshold=1000
)

print("Configuracoes NumPy aplicadas!")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)
pd.set_option('mode.use_inf_as_na', True)

print("Configuracoes Pandas aplicadas!")

# Configurar Plotly
pio.templates.default = "plotly_white"
pio.renderers.default = "browser"

print("Plotly configurado!")

Diretorio atual: c:\Users\caroline.coutinho\projeto_mediacao_bancaria\analises\gold
Raiz do projeto: c:\Users\caroline.coutinho\projeto_mediacao_bancaria
Caminho adicionado ao sys.path

Pasta 'lib' encontrada em: c:\Users\caroline.coutinho\projeto_mediacao_bancaria\lib

Bibliotecas basicas importadas!
Modulos customizados importados!
Configuracoes NumPy aplicadas!
Configuracoes Pandas aplicadas!
Plotly configurado!


In [51]:
print("\nAplicando tema Agibank...")

# Aplicar tema customizado
aplicar_tema_agibank()
configurar_plotly()

print("OK - Tema Agibank aplicado!")

# Exibir cores disponiveis
print("\nCores Agibank disponiveis:")
for nome, cor in CORES_AGIBANK.items():
    print(f"  {nome}: {cor}")

print("\nPaletas disponiveis:")
print(f"  PALETA_CATEGORICA: {len(PALETA_CATEGORICA)} cores")
print(f"  PALETA_AZUL: {len(PALETA_AZUL)} tons")
print(f"  PALETA_VERDE: {len(PALETA_VERDE)} tons")


Aplicando tema Agibank...
Tema Agibank aplicado - Tamanho: medio
Tema Agibank aplicado ao Plotly
OK - Tema Agibank aplicado!

Cores Agibank disponiveis:
  azul_principal: #0064f5
  azul_medio: #0053b0
  azul_escuro: #000f44
  verde: #77df40
  verde_claro: #c5ff90
  amarelo: #ffd600
  branco: #ffffff

Paletas disponiveis:
  PALETA_CATEGORICA: 6 cores
  PALETA_AZUL: 3 tons
  PALETA_VERDE: 2 tons


In [52]:
print("\nCriando funcoes auxiliares adicionais...")

# Primeiro, vamos ver quais cores estao disponiveis
print("\nCores disponiveis em CORES_AGIBANK:")
for chave in CORES_AGIBANK.keys():
    print(f"  - {chave}")

# Pegar a primeira cor disponivel como padrao
COR_PADRAO = list(CORES_AGIBANK.values())[0]

def formatar_numero(valor, tipo='inteiro'):
    """Formata numeros para exibicao"""
    if pd.isna(valor):
        return 'N/A'
    
    if tipo == 'inteiro':
        return f"{int(valor):,}".replace(',', '.')
    elif tipo == 'decimal':
        return f"{valor:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
    elif tipo == 'percentual':
        return f"{valor:.2f}%"
    elif tipo == 'moeda':
        return f"R$ {valor:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')
    else:
        return str(valor)

def criar_anotacao(texto, x, y, cor=None):
    """Cria anotacao para graficos"""
    if cor is None:
        cor = COR_PADRAO
    
    return dict(
        x=x, y=y, text=texto,
        showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=2, arrowcolor=cor,
        ax=20, ay=-30,
        font=dict(size=10, color=cor),
        bgcolor='white', bordercolor=cor, borderwidth=1
    )

def salvar_grafico(fig, nome_arquivo, pasta='graficos_estrategicos'):
    """Salva grafico em HTML e PNG"""
    caminho_pasta = Path(pasta)
    caminho_pasta.mkdir(exist_ok=True)
    
    caminho_html = caminho_pasta / f"{nome_arquivo}.html"
    fig.write_html(caminho_html)
    
    try:
        caminho_png = caminho_pasta / f"{nome_arquivo}.png"
        fig.write_image(caminho_png, width=1200, height=800)
        print(f"OK - Grafico salvo: {nome_arquivo}.html e .png")
    except:
        print(f"OK - Grafico salvo: {nome_arquivo}.html (PNG requer kaleido)")

def criar_tabela_comparativa(df, colunas, titulo):
    """Cria tabela formatada"""
    # Usar primeira cor do dicionario para header
    cor_header = list(CORES_AGIBANK.values())[0] if len(CORES_AGIBANK) > 0 else '#004E89'
    
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=[f"<b>{col}</b>" for col in colunas],
            fill_color=cor_header,
            font=dict(color='white', size=12),
            align='left'
        ),
        cells=dict(
            values=[df[col] for col in colunas],
            fill_color='white',
            font=dict(size=11),
            align='left',
            height=25
        )
    )])
    
    fig.update_layout(
        title=titulo,
        height=400 + len(df) * 25,
        margin=dict(l=20, r=20, t=60, b=20)
    )
    return fig

print("OK - Funcoes auxiliares criadas!")
print("\nFuncoes disponiveis:")
print("  - formatar_numero()")
print("  - criar_anotacao()")
print("  - salvar_grafico()")
print("  - criar_tabela_comparativa()")
print("  + Funcoes da lib.visualizacoes")


Criando funcoes auxiliares adicionais...

Cores disponiveis em CORES_AGIBANK:
  - azul_principal
  - azul_medio
  - azul_escuro
  - verde
  - verde_claro
  - amarelo
  - branco
OK - Funcoes auxiliares criadas!

Funcoes disponiveis:
  - formatar_numero()
  - criar_anotacao()
  - salvar_grafico()
  - criar_tabela_comparativa()
  + Funcoes da lib.visualizacoes


In [53]:
print("=" * 100)
print("CARREGANDO DADOS - SETOR BANCARIO")
print("=" * 100)

# Caminho dos dados
CAMINHO_BANCARIO = Path('../../data/gold/dados_prontos_plotagem/setor_bancario')

print(f"Caminho: {CAMINHO_BANCARIO.absolute()}")

# Carregar CSVs
df_estados_bancario = pd.read_csv(CAMINHO_BANCARIO / 'estados_bancario.csv')
df_municipios_sp_bancario = pd.read_csv(CAMINHO_BANCARIO / 'municipios_sp_bancario.csv')
df_municipios_agibank = pd.read_csv(CAMINHO_BANCARIO / 'municipios_agibank.csv')
df_instituicoes = pd.read_csv(CAMINHO_BANCARIO / 'instituicoes_bancarias_sp.csv')

print(f"\nOK - Estados: {len(df_estados_bancario)}")
print(f"OK - Municipios SP: {len(df_municipios_sp_bancario)}")
print(f"OK - Municipios Agibank: {len(df_municipios_agibank)}")
print(f"OK - Instituicoes: {len(df_instituicoes)}")

print("\n" + "=" * 100)
print("DADOS CARREGADOS!")
print("=" * 100)

CARREGANDO DADOS - SETOR BANCARIO
Caminho: c:\Users\caroline.coutinho\projeto_mediacao_bancaria\analises\gold\..\..\data\gold\dados_prontos_plotagem\setor_bancario

OK - Estados: 27
OK - Municipios SP: 634
OK - Municipios Agibank: 361
OK - Instituicoes: 293

DADOS CARREGADOS!


In [54]:
print("=" * 100)
print("PREVIEW DOS DADOS")
print("=" * 100)

print("\n1. ESTADOS:")
display(df_estados_bancario.head(3))

print("\n2. MUNICIPIOS SP:")
display(df_municipios_sp_bancario.head(3))

print("\n3. AGIBANK:")
display(df_municipios_agibank.head(3))

print("\n4. BANCOS:")
display(df_instituicoes.head(5))

PREVIEW DOS DADOS

1. ESTADOS:


,uf,regiao,nota_media,tempo_medio,populacao,total_reclamacoes,reclamacoes_100k,pct_resolvido
0,AC,Norte,2.41,5.92,830018,3359,404.69,8.37
1,AL,Nordeste,2.29,5.96,3127683,9455,302.30,7.41
2,AM,Norte,2.30,6.47,3941613,18062,458.24,6.61



2. MUNICIPIOS SP:


,municipio,total_reclamacoes,populacao,nota_media,tempo_medio,pct_resolvido,reclamacoes_100k
0,ADAMANTINA,156,34687.00,1.60,6.16,1.92,449.74
1,ADOLFO,9,4351.00,5.00,5.00,0.00,206.85
2,AGUAÍ,83,32072.00,3.36,5.57,10.84,258.79



3. AGIBANK:


,municipio,total_reclamacoes,populacao,nota_media,tempo_medio,pct_resolvido,reclamacoes_100k
0,ADAMANTINA,2,34687.00,NaN,6.50,0.00,5.77
1,AGUAÍ,1,32072.00,NaN,NaN,0.00,3.12
2,AGUDOS,1,37680.00,1.00,8.00,0.00,2.65



4. BANCOS:


,instituicao,total_reclamacoes,nota_media,tempo_medio,pct_resolvido,segmento
0,Nubank,37381,1.80,3.47,3.52,"Bancos, Financeiras e Administradoras de Cartão"
1,Banco Santander,26741,2.07,5.68,7.05,"Bancos, Financeiras e Administradoras de Cartão"
2,Banco Bradesco,20734,2.03,8.76,5.60,"Bancos, Financeiras e Administradoras de Cartão"
3,Banco do Brasil,17929,2.47,4.59,7.88,"Bancos, Financeiras e Administradoras de Cartão"
4,Banco Itaú Unibanco,17660,2.08,6.74,7.17,"Bancos, Financeiras e Administradoras de Cartão"


In [55]:
print("=" * 100)
print("NUMEROS-CHAVE - SETOR BANCARIO")
print("=" * 100)

# Brasil
total_brasil = df_estados_bancario['total_reclamacoes'].sum()
nota_brasil = df_estados_bancario['nota_media'].mean()
tempo_brasil = df_estados_bancario['tempo_medio'].mean()
resol_brasil = df_estados_bancario['pct_resolvido'].mean()

# SP
sp = df_estados_bancario[df_estados_bancario['uf'] == 'SP'].iloc[0]

# Agibank
total_agibank = df_municipios_agibank['total_reclamacoes'].sum()
nota_agibank = df_municipios_agibank['nota_media'].mean()
tempo_agibank = df_municipios_agibank['tempo_medio'].mean()
resol_agibank = df_municipios_agibank['pct_resolvido'].mean()

# Exibir
print("\nBRASIL:")
print(f"  Total: {total_brasil:,}")
print(f"  Nota: {nota_brasil:.2f}")
print(f"  Tempo: {tempo_brasil:.2f} dias")
print(f"  % Resolvido: {resol_brasil:.2f}%")

print("\nSAO PAULO:")
print(f"  Total: {int(sp['total_reclamacoes']):,}")
print(f"  Nota: {sp['nota_media']:.2f}")
print(f"  Tempo: {sp['tempo_medio']:.2f} dias")
print(f"  % Resolvido: {sp['pct_resolvido']:.2f}%")

print("\nAGIBANK:")
print(f"  Total: {total_agibank:,}")
print(f"  Nota: {nota_agibank:.2f}")
print(f"  Tempo: {tempo_agibank:.2f} dias")
print(f"  % Resolvido: {resol_agibank:.2f}%")

print("\nCOMPARACAO:")
print(f"  Nota: {nota_agibank - nota_brasil:+.2f} {'(pior)' if nota_agibank < nota_brasil else '(melhor)'}")
print(f"  Tempo: {tempo_agibank - tempo_brasil:+.2f} dias {'(mais lento)' if tempo_agibank > tempo_brasil else '(mais rapido)'}")
print(f"  % Resolvido: {resol_agibank - resol_brasil:+.2f}% {'(pior)' if resol_agibank < resol_brasil else '(melhor)'}")

print("\n" + "=" * 100)
print("PRONTO PARA GRAFICOS!")
print("=" * 100)

NUMEROS-CHAVE - SETOR BANCARIO

BRASIL:
  Total: 1,109,438
  Nota: 2.30
  Tempo: 6.16 dias
  % Resolvido: 7.16%

SAO PAULO:
  Total: 276,735
  Nota: 2.19
  Tempo: 6.27 dias
  % Resolvido: 7.21%

AGIBANK:
  Total: 3,969
  Nota: 1.87
  Tempo: 6.75 dias
  % Resolvido: 6.75%

COMPARACAO:
  Nota: -0.43 (pior)
  Tempo: +0.59 dias (mais lento)
  % Resolvido: -0.41% (pior)

PRONTO PARA GRAFICOS!


In [56]:
print("=" * 100)
print("VALIDACAO: CAMPINAS - MERCADO vs AGIBANK")
print("=" * 100)

# Campinas - Mercado (todos os bancos)
campinas_mercado = df_municipios_sp_bancario[
    df_municipios_sp_bancario['municipio'] == 'CAMPINAS'
]

# Campinas - Agibank
campinas_agibank = df_municipios_agibank[
    df_municipios_agibank['municipio'] == 'CAMPINAS'
]

print("\nCAMPINAS - MERCADO BANCARIO (todos os bancos):")
if len(campinas_mercado) > 0:
    c_merc = campinas_mercado.iloc[0]
    print(f"  Total reclamacoes:  {int(c_merc['total_reclamacoes']):,}")
    print(f"  Populacao:          {int(c_merc['populacao']):,}")
    print(f"  Nota media:         {c_merc['nota_media']:.2f}")
    print(f"  Tempo medio:        {c_merc['tempo_medio']:.2f} dias")
    print(f"  % Resolvido:        {c_merc['pct_resolvido']:.2f}%")
else:
    print("  NAO ENCONTRADO!")

print("\nCAMPINAS - AGIBANK:")
if len(campinas_agibank) > 0:
    c_agi = campinas_agibank.iloc[0]
    print(f"  Total reclamacoes:  {int(c_agi['total_reclamacoes']):,}")
    print(f"  Populacao:          {int(c_agi['populacao']):,}")
    print(f"  Nota media:         {c_agi['nota_media']:.2f}")
    print(f"  Tempo medio:        {c_agi['tempo_medio']:.2f} dias")
    print(f"  % Resolvido:        {c_agi['pct_resolvido']:.2f}%")
else:
    print("  NAO ENCONTRADO!")

if len(campinas_mercado) > 0 and len(campinas_agibank) > 0:
    print("\n" + "=" * 100)
    print("COMPARACAO CAMPINAS:")
    print("=" * 100)
    
    print(f"\nVolume:")
    print(f"  Mercado:  {int(c_merc['total_reclamacoes']):>8,}")
    print(f"  Agibank:  {int(c_agi['total_reclamacoes']):>8,}")
    print(f"  % Agibank: {(c_agi['total_reclamacoes']/c_merc['total_reclamacoes']*100):>7.2f}%")
    
    print(f"\nQualidade:")
    print(f"                Mercado    Agibank    Diferenca")
    print(f"  Nota:         {c_merc['nota_media']:>7.2f}    {c_agi['nota_media']:>7.2f}    {c_agi['nota_media']-c_merc['nota_media']:>+7.2f}")
    print(f"  Tempo:        {c_merc['tempo_medio']:>7.2f}    {c_agi['tempo_medio']:>7.2f}    {c_agi['tempo_medio']-c_merc['tempo_medio']:>+7.2f} dias")
    print(f"  % Resolvido:  {c_merc['pct_resolvido']:>7.2f}    {c_agi['pct_resolvido']:>7.2f}    {c_agi['pct_resolvido']-c_merc['pct_resolvido']:>+7.2f}%")

print("\n" + "=" * 100)

VALIDACAO: CAMPINAS - MERCADO vs AGIBANK

CAMPINAS - MERCADO BANCARIO (todos os bancos):
  Total reclamacoes:  7,606
  Populacao:          1,139,047
  Nota media:         2.15
  Tempo medio:        6.27 dias
  % Resolvido:        7.43%

CAMPINAS - AGIBANK:
  Total reclamacoes:  78
  Populacao:          1,139,047
  Nota media:         1.92
  Tempo medio:        6.59 dias
  % Resolvido:        5.13%

COMPARACAO CAMPINAS:

Volume:
  Mercado:     7,606
  Agibank:        78
  % Agibank:    1.03%

Qualidade:
                Mercado    Agibank    Diferenca
  Nota:            2.15       1.92      -0.23
  Tempo:           6.27       6.59      +0.32 dias
  % Resolvido:     7.43       5.13      -2.30%



In [58]:
# GRAFICO 1: MAPA DE CALOR BRASIL - SETOR BANCARIO 


print("\n" + "=" * 80)
print("GRAFICO 1: MAPA DE CALOR BRASIL - SETOR BANCARIO")
print("=" * 80)

# ETAPA 1: PREPARAÇÃO DE DADOS


print("\n[1/6] Preparando dados com NumPy...")

# Converter para arrays NumPy
uf = df_estados_bancario['uf'].values
regiao = df_estados_bancario['regiao'].values
total_reclamacoes = df_estados_bancario['total_reclamacoes'].values.astype(np.float64)
populacao = df_estados_bancario['populacao'].values.astype(np.float64)
reclamacoes_100k = df_estados_bancario['reclamacoes_100k'].values.astype(np.float64)
nota_media = df_estados_bancario['nota_media'].values.astype(np.float64)
tempo_medio = df_estados_bancario['tempo_medio'].values.astype(np.float64)
pct_resolvido = df_estados_bancario['pct_resolvido'].values.astype(np.float64)

print(f"   Arrays NumPy criados: {len(uf)} estados")

# ============================================================================
# ETAPA 2: CARREGAR GEOJSON DO BRASIL
# ============================================================================

print("\n[2/6] Carregando GeoJSON do Brasil...")

url_geojson = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson'

try:
    with urllib.request.urlopen(url_geojson) as response:
        geojson_brasil = json.loads(response.read())
    print("   GeoJSON carregado com sucesso!")
except:
    print("   Erro ao carregar GeoJSON online")
    geojson_brasil = None

# ============================================================================
# ETAPA 3: CALCULOS ESTATISTICOS COM NUMPY
# ============================================================================

print("\n[3/6] Calculando estatísticas...")

media_taxa = np.mean(reclamacoes_100k)
mediana_taxa = np.median(reclamacoes_100k)
desvio_taxa = np.std(reclamacoes_100k)
max_taxa = np.max(reclamacoes_100k)
min_taxa = np.min(reclamacoes_100k)

idx_max = np.argmax(reclamacoes_100k)
idx_min = np.argmin(reclamacoes_100k)

# Calcular quartis para legenda
q1 = np.percentile(reclamacoes_100k, 25)
q2 = np.percentile(reclamacoes_100k, 50)
q3 = np.percentile(reclamacoes_100k, 75)

print(f"   Média: {media_taxa:.0f}/100k")
print(f"   Q1: {q1:.0f} | Q2: {q2:.0f} | Q3: {q3:.0f}")

# Classificar estados por criticidade (NumPy)
criticidade = np.where(
    reclamacoes_100k > q3, 'ALTO',
    np.where(reclamacoes_100k > q2, 'MODERADO-ALTO',
    np.where(reclamacoes_100k > q1, 'MODERADO-BAIXO', 'BAIXO'))
)

# ============================================================================
# ETAPA 4: PREPARAR CUSTOMDATA
# ============================================================================

print("\n[4/6] Preparando dados para visualização...")

# Criar ranking (NumPy)
ranking = np.argsort(np.argsort(reclamacoes_100k)[::-1]) + 1

customdata_matrix = np.column_stack([
    total_reclamacoes,
    populacao,
    nota_media,
    tempo_medio,
    pct_resolvido,
    ranking
])

# ============================================================================
# ETAPA 5: CRIAR MAPA COM CORES AGIBANK MELHORADAS
# ============================================================================

print("\n[5/6] Gerando mapa interativo...")

fig = go.Figure()

if geojson_brasil:
    # Escala de cores usando TODAS as cores Agibank
    colorscale_agibank = [
        [0.0, CORES_AGIBANK['verde_claro']],      # Muito baixo
        [0.15, CORES_AGIBANK['verde']],           # Baixo
        [0.30, CORES_AGIBANK['amarelo']],         # Moderado-baixo
        [0.50, '#FFA500'],                        # Laranja (transição)
        [0.65, CORES_AGIBANK['azul_principal']],  # Moderado-alto
        [0.80, CORES_AGIBANK['azul_medio']],      # Alto
        [1.0, CORES_AGIBANK['azul_escuro']]       # Muito alto
    ]
    
    fig.add_trace(go.Choropleth(
        geojson=geojson_brasil,
        locations=uf,
        z=reclamacoes_100k,
        featureidkey="properties.sigla",
        colorscale=colorscale_agibank,
        text=uf,
        marker_line_color='white',
        marker_line_width=2,
        zmin=min_taxa,
        zmax=max_taxa,
        colorbar=dict(
            title=dict(
                text='<b>Taxa/100k hab</b>',
                font=dict(size=14, color='#1a1a1a', family='Arial')
            ),
            tickfont=dict(size=12, color='#333333'),
            len=0.7,
            thickness=20,
            x=1.02,
            bgcolor='rgba(255,255,255,0.9)',
            bordercolor='#CCCCCC',
            borderwidth=2,
            tickmode='array',
            tickvals=[min_taxa, q1, q2, q3, max_taxa],
            ticktext=[
                f'{min_taxa:.0f}<br><span style="font-size:9px">Mínimo</span>',
                f'{q1:.0f}<br><span style="font-size:9px">Q1</span>',
                f'{q2:.0f}<br><span style="font-size:9px">Mediana</span>',
                f'{q3:.0f}<br><span style="font-size:9px">Q3</span>',
                f'{max_taxa:.0f}<br><span style="font-size:9px">Máximo</span>'
            ]
        ),
        hovertemplate=(
            '<b style="font-size:20px; color:#000f44">%{text}</b><br>'
            '<span style="font-size:11px; color:#999999">Ranking: %{customdata[5]}º de 27</span>'
            '<br><br>'
            '<span style="color:#dddddd">──────────────────</span><br>'
            '<span style="font-size:13px; color:#666666">TAXA NORMALIZADA</span><br>'
            '<span style="font-size:24px; color:#000f44"><b>%{z:.0f}</b></span> '
            '<span style="font-size:12px; color:#777777">por 100k hab</span>'
            '<br><br>'
            '<span style="color:#dddddd">──────────────────</span><br>'
            '<span style="font-size:13px; color:#666666">VOLUME TOTAL</span><br>'
            '<span style="font-size:18px; color:#000f44"><b>%{customdata[0]:,}</b></span><br>'
            '<span style="font-size:11px; color:#777777">reclamações</span>'
            '<br><br>'
            '<span style="color:#dddddd">──────────────────</span><br>'
            '<span style="font-size:13px; color:#666666">POPULAÇÃO</span><br>'
            '<span style="font-size:16px"><b>%{customdata[1]:,}</b></span> hab'
            '<br><br>'
            '<span style="color:#dddddd">──────────────────</span><br>'
            '<span style="font-size:12px; color:#666666">INDICADORES</span><br>'
            'Nota: <b>%{customdata[2]:.2f}</b>/5.0<br>'
            'Tempo: <b>%{customdata[3]:.1f}</b> dias<br>'
            'Resolvido: <b>%{customdata[4]:.1f}%</b>'
            '<br><br>'
            '<extra></extra>'
        ),
        customdata=customdata_matrix
    ))

fig.update_geos(
    fitbounds="locations",
    visible=False,
    bgcolor='#F8F9FA'
)

fig.update_layout(
    title={
        'text': (
            f'<b style="color:{CORES_AGIBANK["azul_escuro"]}; font-size:26px">Gráfico 1: Mapa de Calor - Setor Bancário Brasil</b><br>'
            '<span style="font-size:15px; color:#666666">Taxa de Reclamações por 100 mil habitantes (2025)</span><br>'
            '<span style="font-size:12px; color:#999999">Fonte: Consumidor.gov.br | Análise: Projeto Agibank</span>'
        ),
        'x': 0.5,
        'xanchor': 'center',
        'font': {'family': 'Arial'}
    },
    height=750,
    paper_bgcolor='white',
    margin=dict(t=120, l=10, r=180, b=10),
    hoverlabel=dict(
        bgcolor='rgba(255, 255, 255, 0.98)',
        font_size=13,
        font_family='Arial',
        bordercolor=CORES_AGIBANK['azul_escuro'],
        align='left'
    )
)

# ============================================================================
# LEGENDA PRINCIPAL (SUPERIOR ESQUERDO)
# ============================================================================

fig.add_annotation(
    text=(
        f'<b style="color:{CORES_AGIBANK["azul_escuro"]}; font-size:16px">ESTATÍSTICAS BRASIL</b><br><br>'
        
        f'<span style="font-size:13px; color:#333333">'
        f'<b>Total de Reclamações:</b><br>'
        f'<span style="font-size:20px; color:{CORES_AGIBANK["azul_principal"]}"><b>{int(np.sum(total_reclamacoes)):,}</b></span>'
        f'</span><br><br>'
        
        f'<span style="font-size:12px; color:#555555">'
        f'<b>Taxa Média:</b> {media_taxa:.0f}/100k<br>'
        f'<b>Mediana:</b> {mediana_taxa:.0f}/100k<br>'
        f'<b>Desvio Padrão:</b> ±{desvio_taxa:.0f}<br><br>'
        
        f'<b>Maior Taxa:</b> {uf[idx_max]} ({max_taxa:.0f}/100k)<br>'
        f'<b>Menor Taxa:</b> {uf[idx_min]} ({min_taxa:.0f}/100k)<br><br>'
        
        f'<b>Distribuição por Quartil:</b><br>'
        f'Q1 (25%): até {q1:.0f}/100k<br>'
        f'Q2 (50%): até {q2:.0f}/100k<br>'
        f'Q3 (75%): até {q3:.0f}/100k<br>'
        f'Q4 (100%): até {max_taxa:.0f}/100k'
        f'</span>'
    ),
    xref="paper", yref="paper",
    x=0.01, y=0.99,
    xanchor='left', yanchor='top',
    showarrow=False,
    bgcolor='rgba(255,255,255,0.97)',
    bordercolor=CORES_AGIBANK['azul_principal'],
    borderwidth=3,
    font=dict(color='#1a1a1a', family='Arial'),
    align='left',
    borderpad=15
)

# ============================================================================
# LEGENDA DE CORES (INFERIOR ESQUERDO)
# ============================================================================

fig.add_annotation(
    text=(
        f'<b style="color:{CORES_AGIBANK["azul_escuro"]}; font-size:14px">LEGENDA</b><br><br>'
        
        f'<span style="font-size:12px; color:#333333">'
        f'🟢 <b>Muito Baixo</b> (< {q1:.0f})<br>'
        f'🟡 <b>Baixo</b> ({q1:.0f} - {q2:.0f})<br>'
        f'🟠 <b>Moderado</b> ({q2:.0f} - {q3:.0f})<br>'
        f'🔵 <b>Alto</b> ({q3:.0f} - {max_taxa*.8:.0f})<br>'
        f'🔴 <b>Muito Alto</b> (> {max_taxa*.8:.0f})<br><br>'
        
        f'<span style="font-size:10px; color:#888888">'
        f'Cores: Verde → Amarelo → Azul<br>'
        f'Intensidade indica criticidade'
        f'</span>'
        f'</span>'
    ),
    xref="paper", yref="paper",
    x=0.01, y=0.32,
    xanchor='left', yanchor='top',
    showarrow=False,
    bgcolor='rgba(255,255,255,0.97)',
    bordercolor=CORES_AGIBANK['verde'],
    borderwidth=2,
    font=dict(color='#1a1a1a', family='Arial'),
    align='left',
    borderpad=12
)

# ============================================================================
# RODAPÉ COM INSIGHT
# ============================================================================

fig.add_annotation(
    text=(
        f'<span style="font-size:11px; color:#666666">'
        f'Insight: Estados com maior taxa não necessariamente têm maior volume absoluto. '
        f'A normalização por população permite comparação justa entre regiões de tamanhos diferentes.'
        f'</span>'
    ),
    xref="paper", yref="paper",
    x=0.5, y=-0.02,
    xanchor='center', yanchor='top',
    showarrow=False,
    bgcolor='rgba(255,255,255,0.95)',
    bordercolor='#DDDDDD',
    borderwidth=1,
    font=dict(color='#555555', family='Arial'),
    align='center',
    borderpad=8
)

fig.show()

# ============================================================================
# ETAPA 6: SALVAR
# ============================================================================

print("\n[6/6] Salvando gráfico...")

CAMINHO_GRAFICOS = Path('graficos_estrategicos')
CAMINHO_GRAFICOS.mkdir(exist_ok=True)
fig.write_html(CAMINHO_GRAFICOS / 'grafico_01_mapa_brasil.html')

print(f"✅ Gráfico salvo em: {CAMINHO_GRAFICOS / 'grafico_01_mapa_brasil.html'}")

# ============================================================================
# RESUMO EXECUTIVO COM NUMPY
# ============================================================================

print(f"\n" + "="*80)
print("RESUMO EXECUTIVO - ANÁLISE COM NUMPY")
print("="*80)

# Ranking completo (NumPy)
ranking_indices = np.argsort(reclamacoes_100k)[::-1]

print(f"\nTop 5 Estados - MAIOR Taxa/100k:")
for i, idx in enumerate(ranking_indices[:5], 1):
    print(f"   {i}º {uf[idx]:2s} | {reclamacoes_100k[idx]:>6.0f}/100k | "
          f"{int(total_reclamacoes[idx]):>8,} recl | Nota: {nota_media[idx]:.2f}")

print(f"\nTop 5 Estados - MENOR Taxa/100k:")
for i, idx in enumerate(ranking_indices[-5:][::-1], 1):
    print(f"   {i}º {uf[idx]:2s} | {reclamacoes_100k[idx]:>6.0f}/100k | "
          f"{int(total_reclamacoes[idx]):>8,} recl | Nota: {nota_media[idx]:.2f}")

# Análise por região (NumPy)
print(f"\nAnálise por Região:")
regioes_unicas = np.unique(regiao)
for reg in regioes_unicas:
    mask = regiao == reg
    taxa_media_regiao = np.mean(reclamacoes_100k[mask])
    total_regiao = np.sum(total_reclamacoes[mask])
    qtd_estados = np.sum(mask)
    print(f"   {reg:15s} | Média: {taxa_media_regiao:>6.0f}/100k | "
          f"Total: {int(total_regiao):>8,} | Estados: {qtd_estados}")

# Distribuição por criticidade
print(f"\nDistribuição por Criticidade:")
unique_crit, counts_crit = np.unique(criticidade, return_counts=True)
for crit, count in zip(unique_crit, counts_crit):
    pct = (count / len(uf)) * 100
    print(f"   {crit:15s}: {count:2d} estados ({pct:.1f}%)")

print(f"\n" + "="*80)


GRAFICO 1: MAPA DE CALOR BRASIL - SETOR BANCARIO

[1/6] Preparando dados com NumPy...
   Arrays NumPy criados: 27 estados

[2/6] Carregando GeoJSON do Brasil...
   GeoJSON carregado com sucesso!

[3/6] Calculando estatísticas...
   Média: 514/100k
   Q1: 392 | Q2: 496 | Q3: 616

[4/6] Preparando dados para visualização...

[5/6] Gerando mapa interativo...

[6/6] Salvando gráfico...
✅ Gráfico salvo em: graficos_estrategicos\grafico_01_mapa_brasil.html

RESUMO EXECUTIVO - ANÁLISE COM NUMPY

Top 5 Estados - MAIOR Taxa/100k:
   1º DF |   1209/100k |   34,062 recl | Nota: 2.32
   2º SC |    662/100k |   50,381 recl | Nota: 2.29
   3º MT |    658/100k |   24,089 recl | Nota: 2.32
   4º MG |    643/100k |  132,043 recl | Nota: 2.15
   5º RJ |    641/100k |  102,950 recl | Nota: 2.21

Top 5 Estados - MENOR Taxa/100k:
   1º AL |    302/100k |    9,455 recl | Nota: 2.29
   2º PA |    307/100k |   24,961 recl | Nota: 2.32
   3º PE |    352/100k |   31,893 recl | Nota: 2.25
   4º CE |    362/100k